# Setup directories

In [1]:
import os
derivs_dir = os.path.join('..','derivatives')
homog_dir = os.path.join(derivs_dir,'02.homogenized')
source_dir = os.path.join('..','sourcedata')

# Import dataframes

In [2]:
import pandas as pd
fpath = os.path.join(homog_dir,'econdec-full_task-main_beh.xlsx')
beh_frame = pd.read_excel(fpath)

In [3]:
fpath = os.path.join(source_dir,'raw-gaze-data.xlsx')
choice_gaze_frame = pd.read_excel(fpath, sheet_name=0)

In [4]:
outcome_gaze_frame = pd.read_excel(fpath, sheet_name=1)

# Extract only the gaze data for main task series

The dataframe contains gaze data for practice trials, where `practice == 1` or `practice == 2`.

It also contains gaze data for the Memory task trials, where `Phase == Fract` or `Phase == Face`.

We'll slice all these rows out, leaving only the relevant main task trials (1-72).

In [5]:
choice_gaze_frame = choice_gaze_frame[
    choice_gaze_frame['practice']==3
][
    choice_gaze_frame['Phase'] == 'Main Task'
]

C:\Users\ia406477\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [6]:
outcome_gaze_frame = outcome_gaze_frame[
    outcome_gaze_frame['practice'] == 3
][
    outcome_gaze_frame['Phase'] == 'Main Task'
]

C:\Users\ia406477\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


# Fix gaze trial numbers

They were offset by 4 by the practice trials (1-4) we removed. Re-setting to initialize at 1

In [7]:
choice_gaze_frame['TRIAL_LABEL'] = choice_gaze_frame['TRIAL_LABEL']-4

In [8]:
outcome_gaze_frame['TRIAL_LABEL'] = outcome_gaze_frame['TRIAL_LABEL']-4

# Drop irrelevant columns

There is a lot of data here, so we can implicitly select only the columns we need.

We'll rename them to be a little more clear next.

In [9]:
choice_gaze_frame = choice_gaze_frame[[
    'RECORDING_SESSION_LABEL','TRIAL_LABEL',
    'IA_LABEL','IA_ID',
    'IA_FSA_COUNT_1','IA_FSA_COUNT_2',
    'IA_FSA_COUNT_24','IA_FSA_COUNT_25',
    'IA_FSA_COUNT_26','IA_FSA_COUNT_27',
]]

In [10]:
outcome_gaze_frame = outcome_gaze_frame[[
    'RECORDING_SESSION_LABEL','TRIAL_LABEL',
    'IA_ID','IA_LABEL',
    'IA_DWELL_TIME'
]]

# Rename remaining columns

In the choice phase, we need to keep this entire matrix in order to calculate the oscillation rate. Each row represents an on-screen interest area for a given trial, denoted redundantly with `ia-id` and `ia-label`. There are six(6) relevant interest areas during the choice phase, so there are six(6) rows per trial.

The matrix tells us how many times a saccade started in one interest area and ended in another. Each of these is called a "fixation skip" (FSA), starting in that row's interest area, and ending in the interest area denoted by the `fsa-ia-` columns.

In [11]:
choice_gaze_frame = choice_gaze_frame.rename(columns={
    'RECORDING_SESSION_LABEL':'subjnum',
    'TRIAL_LABEL':'trial',
    'IA_ID':'ia-id',
    'IA_LABEL':'ia-label',
    'IA_DWELL_TIME':'dwell-time',
    'IA_FSA_COUNT_1':'fsa-ia-01',
    'IA_FSA_COUNT_2':'fsa-ia-02',
    'IA_FSA_COUNT_24':'fsa-ia-24',
    'IA_FSA_COUNT_25':'fsa-ia-25',
    'IA_FSA_COUNT_26':'fsa-ia-26',
    'IA_FSA_COUNT_27':'fsa-ia-27'
})

In [12]:
outcome_dwell_time = outcome_gaze_frame.rename(columns={
    'RECORDING_SESSION_LABEL':'subjnum',
    'TRIAL_LABEL':'trial',
    'IA_ID':'ia-id',
    'IA_LABEL':'ia-label',
    'IA_DWELL_TIME':'dwell-time'
})

# Transform matrix into oscillation sum

We'll first define a function `sum_oscillations` to use with `df.apply()` to sum up the number of oscillations from any a given row's interest area to any of the interest areas on the other side of the screen.

In [13]:
stock_ia_list = [1,24,26]
bond_ia_list = [2,25,27]

def sum_oscillations(row):
    if row['ia-id'] in stock_ia_list:
        return sum([row['fsa-ia-02'],row['fsa-ia-25'],row['fsa-ia-27']])
    elif row['ia-id'] in bond_ia_list:
        return sum([row['fsa-ia-01'],row['fsa-ia-24'],row['fsa-ia-26']])
    
choice_gaze_frame['oscillations'] = choice_gaze_frame.apply(sum_oscillations,axis=1)

Next we use `df.groupby()` and `df.sum()` to collect and summate the oscillations into trialwise rows for merging into `beh_frame`

In [14]:
oscillations = choice_gaze_frame.groupby(['subjnum','trial'])
oscillations = oscillations.sum()['oscillations']
oscillations = oscillations.reset_index()

In [15]:
beh_frame = beh_frame.merge(oscillations,'left')

# Extract outcome "Bubble" dwell time

All we need here is the `dwell-time` for one interest area in particular (`ia-id == 5`)

In [16]:
outcome_dwell_time = outcome_dwell_time[outcome_dwell_time['ia-id']==5]

In [17]:
outcome_dwell_time = outcome_dwell_time[['subjnum','trial','dwell-time']]

In [18]:
beh_frame = beh_frame.merge(outcome_dwell_time,'left')

# Calculate oscillation rate

Divide the sum count of oscillations by the number of seconds spent on the choice phase

In [19]:
def oscillation_rate(row):
    return float(row['oscillations'] / row['choicert'])

In [20]:
beh_frame['osc-rate'] = beh_frame.apply(oscillation_rate,axis=1)

# Output

In [22]:
gaze_dir = os.path.join(derivs_dir,'03.gaze-import')
try: os.mkdir(gaze_dir)
except OSError as e:
    print e

[Error 183] Cannot create a file when that file already exists: '..\\derivatives\\03.gaze-import'


In [23]:
fpath = os.path.join(gaze_dir,'econdec-full_task-all_eye.xlsx')

In [24]:
beh_frame.to_excel(fpath,index=False)